In [1]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
from gurobipy import Model, GRB,QuadExpr,quicksum,LinExpr

In [2]:
def SolveKnapsack(inputfile, method):
    
    [n,c,w,W] = read_instance(inputfile)
    
    if method==1:
        
        start=time.perf_counter()
        optimal_value=Gurobi_Solve(n,c,w,W)
        end=time.perf_counter()
        
        print("Method {}\n".format(method))
        print("Optimal Value: {}".format(optimal_value))
        print("Solve Time: {}\n".format(end-start))
        
#         return end-start
        
    elif method==2:
        
        start1=time.perf_counter()
        all_layers=Algorithm_1(n,c,w,W)
        end1=time.perf_counter()
        
        [maxWidth, nodeCount] = Max_Width(all_layers)
        arcCount = Arc_Count(all_layers)
        
        start2=time.perf_counter()
        optimal_value=Longest_Path(all_layers,c)
        end2=time.perf_counter()
        
        print("Method {}\n".format(method))
        print("Optimal Value: {}".format(optimal_value))
        print("Number of Nodes: {} nodes".format(nodeCount))
        print("Number of Arcs: {} arcs".format(arcCount))
        print("BDD Width: {} nodes".format(maxWidth))
        print("BDD Construction Time: {}s".format(end1-start1))
        print("Solution Time: {}s".format(end2-start2))
        print("Total Time: {}s\n".format((end2-start2)+(end1-start1)))
        
#         return (end2-start2)+(end1-start1)
        
    elif method==3:
        
        start1=time.perf_counter()
        all_layers=Algorithm_1(n,c,w,W)
        end1=time.perf_counter()
        
        start2=time.perf_counter()
        reduced_layers=Algorithm_2(all_layers,n)
        end2=time.perf_counter()
        
        [maxWidth, nodeCount] = Max_Width(reduced_layers)
        arcCount = Arc_Count(reduced_layers)
        
        start3=time.perf_counter()
        optimal_value=Longest_Path(reduced_layers,c)
        end3=time.perf_counter()
        
        print("Method {}\n".format(method))
        print("Optimal Value: {}".format(optimal_value))
        print("Number of Nodes: {} nodes".format(nodeCount))
        print("Number of Arcs: {} arcs".format(arcCount))
        print("BDD Width: {} nodes".format(maxWidth))
        print("BDD Construction Time: {}s".format(end1-start1))
        print("BDD Reduction Time: {}s".format(end2-start2))
        print("Solution Time: {}s".format(end3-start3))
        print("Total Time: {}s\n".format((end3-start3)+(end2-start2)+(end1-start1)))
        
#         return (end3-start3)+(end2-start2)+(end1-start1)
        
    elif method==4:
        
        start0=time.perf_counter()
        c_order=[x for _,x in sorted(zip(w,c))]
        w_order=np.sort(w)
        end0=time.perf_counter()
        
        start1=time.perf_counter()
        all_layers=Algorithm_1(n,c_order,w_order,W)
        end1=time.perf_counter()
        
        [maxWidth, nodeCount] = Max_Width(all_layers)
        arcCount = Arc_Count(all_layers)
        
        start2=time.perf_counter()
        optimal_value=Longest_Path(all_layers,c_order)
        end2=time.perf_counter()
        
        print("Method {}\n".format(method))
        print("Optimal Value: {}".format(optimal_value))
        print("Number of Nodes: {} nodes".format(nodeCount))
        print("Number of Arcs: {} arcs".format(arcCount))
        print("BDD Width: {} nodes".format(maxWidth))
        print("Sorting Time : {}s".format(end0-start0))
        print("BDD Construction Time: {}s".format(end1-start1))
        print("BDD Reduction Time: {}s".format(0))
        print("Solution Time: {}s".format(end2-start2))
        print("Total Time: {}s\n".format((end2-start2)+(end1-start1)+(end0-start0)))
        
#         return (end2-start2)+(end1-start1)+(end0-start0)
        

In [3]:
def read_instance(inputfile):
    data = pd.read_csv(inputfile)
    c = data['c'].values
    w = data['w'].values
    W = data['W'].values[0]
    n = len(c)
    return n,c,w,W

In [4]:
class node(object):
    
    def __init__(self,state):
        self.state=state
        self.inbound={}
        self.outbound={}
        self.cost=0
        
    def __str__(self):
        string="State: "+str(self.state)
        return string

In [5]:
def Gurobi_Solve(n,c,w,W):
    
    m = Model('Binary Tree Search')
    
    m.setParam("OutputFlag", False)

    x = m.addVars(n,vtype = GRB.BINARY,name = "x")

    obj = QuadExpr(quicksum ([x[j] * c[j] for j in range(n)]))

    m.setObjective(obj,GRB.MAXIMIZE)

    m.addConstr(LinExpr(quicksum([w[j] * x[j] for j in range(n)])),GRB.LESS_EQUAL,W)

    m.update()
    m.optimize()
    optimal_x = [x[i].X for i in range(n)]
    optimal_value = m.objVal
    
    return optimal_value

In [6]:
def Algorithm_1(n,cvals,wvals,W):
    
    #All layers is the tree containing each layer
    #All_layers keys are the layer numbers
    #Make root node before beginning
    all_layers={}
    all_layers[0]={}
    all_layers[0][0]=node(0)
    
    #Iterate through each layer
    for x in range(1,n):
        #Create next layer
        currLayer=all_layers[x-1]
        all_layers[x]={}
        nextLayer=all_layers[x]
        
        #Initialize layer variables
        #Saves having to compute this for each node in the layer
        w_current=wvals[x-1]
        c_current=cvals[x-1]
        
        #Begin with the 0 decision: deciding not to take the item
        #Node weight remains the same, so create new node in the subsequent layer
        #Direct arcs to new node
        for nodeInstance in currLayer.values():
            currWeight=nodeInstance.state
            new_node=node(currWeight)
            nextLayer[currWeight]=new_node
            nodeInstance.outbound[0]=new_node
            new_node.inbound[0]=nodeInstance
        
        #Next do the 1 decision: deciding to take the item
        for nodeInstance in currLayer.values():
            newstate=nodeInstance.state+w_current
            #Feasability check
            if newstate<=W:
                #Check if the new state already exists in the subsequent layer
                if nextLayer.get(newstate):
                    #Add arc into node inbound and outbound
                    nodeInstance.outbound[c_current]=nextLayer[newstate]
                    nextLayer[newstate].inbound[c_current]=nodeInstance
                        
                else:
                    #Create new node
                    new_node=node(newstate)
                    nextLayer[newstate]=new_node
                    #Add arc to node inbound and outbound
                    nodeInstance.outbound[c_current]=new_node
                    new_node.inbound[c_current]=nodeInstance
                        
    #Changed from W because Rachel said so
    terminal_node=node(-1)
    all_layers[n]={}
    all_layers[n][-1]=terminal_node
    
    w_final=wvals[n-1]
    c_final=cvals[n-1]
    
    #Swap arc dictionary keys and values
    #This method still leads to probelms in the inbound nodes as you can have multiple inbound arc costs
    #But hey, it works for outbound
    
    #Final layer 0 decision
    for nodeInstance in all_layers[n-1].values():
        
        nodeInstance.outbound[0]=terminal_node
        terminal_node.inbound[0]=nodeInstance
    
    #Final layer 1 decision
    for nodeInstance in all_layers[n-1].values():
        
        if nodeInstance.state+w_final<=W:
            nodeInstance.outbound[c_final]=terminal_node
            terminal_node.inbound[c_final]=nodeInstance
            
    return all_layers

In [7]:
def Algorithm_2(reduced_layers,n):
    #Final reduction algorithm here we go
    #Starting from the second last layer moving up
    for x in range(n-1,0,-1):
        to_remove={}
        tail_list={}
        for nodeInstance in reduced_layers[x].values():
            #Add ending combination if not already in the tail list
            current_tuple=tuple(nodeInstance.outbound.items())
            
            #If ending combination exists
            if tail_list.get(current_tuple):
                to_remove[nodeInstance.state]=nodeInstance
                #Redirect arcs
                for arcCost, comingFrom in nodeInstance.inbound.items():
                    tail_list[current_tuple].inbound[arcCost]=comingFrom
                    comingFrom.outbound[arcCost]=tail_list[current_tuple]
                    
                    #Remove arcs from comingFrom node
                    if comingFrom.outbound.get(nodeInstance):
                        del(comingFrom.outbound[nodeInstance])
            
            #If ending combination does not exist
            else:
                tail_list[current_tuple]=nodeInstance
                        
        #Remove all duplicate nodes            
        for remove_node in to_remove.values():
            del(reduced_layers[x][remove_node.state])
            remove_node=None
        
    return reduced_layers

In [8]:
def Algorithm_2_Hybrid(reduced_layers,n):
    #Final reduction algorithm here we go
    #Starting from the second last layer moving up
    for x in range(n-1,round(2*n/5),-1):
        to_remove={}
        tail_list={}
        for nodeInstance in reduced_layers[x].values():
            #Add ending combination if not already in the tail list
            current_tuple=tuple(nodeInstance.outbound.items())
            
            #If ending combination exists
            if tail_list.get(current_tuple):
                to_remove[nodeInstance.state]=nodeInstance
                #Redirect arcs
                for arcCost, comingFrom in nodeInstance.inbound.items():
                    tail_list[current_tuple].inbound[arcCost]=comingFrom
                    comingFrom.outbound[arcCost]=tail_list[current_tuple]
                    
                    #Remove arcs from comingFrom node
                    if comingFrom.outbound.get(nodeInstance):
                        del(comingFrom.outbound[nodeInstance])
            
            #If ending combination does not exist
            else:
                tail_list[current_tuple]=nodeInstance
                        
        #Remove all duplicate nodes            
        for remove_node in to_remove.values():
            del(reduced_layers[x][remove_node.state])
            remove_node=None
        
    return reduced_layers

In [9]:
def Longest_Path(reduced_layers,cvals):
    #Initiate final score and trail list
    n=len(reduced_layers)-1
    final_score=0
    
    #Iterate through every node and incoming arc, update highest cost found
    for layer in reduced_layers.values():
        for nodeInstance in layer.values():
            for arcCost, goingTo in nodeInstance.outbound.items():
                #Only update arcs where cost > 0
                if (nodeInstance.cost+arcCost)>goingTo.cost:
                    goingTo.cost=nodeInstance.cost+arcCost
                    
    #In the last layer of nodes find the node with the largest cost and set it as the final score
    for nodeInstance in reduced_layers[n].values():
        final_score=nodeInstance.cost
        break
            
    return final_score

In [10]:
def Max_Width(all_layers):
    
    max_width=0
    node_count=0
    
    for layer in all_layers.values():
        no_of_nodes=len(layer)
        node_count+=no_of_nodes
        if no_of_nodes>max_width:
            max_width=no_of_nodes
            
    return max_width, node_count

In [11]:
def Arc_Count(all_layers):
    
    arc_count=0
    
    for layer in all_layers.values():
        for nodeInstance in layer.values():
            arc_count+=len(nodeInstance.outbound)
    
    return arc_count

In [12]:
SolveKnapsack('instance2.csv',1)
SolveKnapsack('instance2.csv',4)
SolveKnapsack('instance2.csv',3)
SolveKnapsack('instance2.csv',2)

Academic license - for non-commercial use only
Method 1

Optimal Value: 2477.0
Solve Time: 0.10101230999999977

Method 4

Optimal Value: 2477
Number of Nodes: 264869 nodes
Number of Arcs: 528178 arcs
BDD Width: 1654 nodes
Sorting Time : 0.0012470410000000598s
BDD Construction Time: 1.5525363159999999s
BDD Reduction Time: 0s
Solution Time: 0.6772187820000006s
Total Time: 2.2310021390000006s

Method 3

Optimal Value: 2477
Number of Nodes: 237376 nodes
Number of Arcs: 473249 arcs
BDD Width: 1559 nodes
BDD Construction Time: 2.2754891419999996s
BDD Reduction Time: 1.2582112209999998s
Solution Time: 0.5870993440000003s
Total Time: 4.120799707s

Method 2

Optimal Value: 2477
Number of Nodes: 360049 nodes
Number of Arcs: 718538 arcs
BDD Width: 1654 nodes
BDD Construction Time: 2.5736508109999985s
Solution Time: 0.8988275769999987s
Total Time: 3.472478387999997s



In [13]:
# def contest(k):
#     y=[]
#     time_2=[]
#     time_3=[]
#     time_4=[]
#     for i in range(k):
#         y.append(i+1)
#         time_2.append(SolveKnapsack('instance.csv',2))
#         time_3.append(SolveKnapsack('instance.csv',3))
#         time_4.append(SolveKnapsack('instance.csv',4))
#     plt.plot(y,time_2,label="Algo 2")
#     plt.plot(y,time_3,label="Algo 3")
#     plt.plot(y,time_4,label="Algo 4")
#     plt.legend()
#     plt.show()
    

In [14]:
# contest(50)